In [1]:
import json
import torch
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('nq-distilbert-base-v1')

taglist = []
questionlist = []
answerdict = {}

with open('intents_even.json', encoding='utf-8-sig') as file:
    data = json.load(file)
    for qanda in data['intents']:
        taglist.append(qanda['tag'])
        for q in qanda['questions']:
            questionlist.append((qanda['tag'], q))
        for a in qanda['answers']:
            answerdict.update({qanda['tag']: a})

In [2]:
questionembeddinglist = [(tagandquestion[0], model.encode(tagandquestion[1])) for tagandquestion in questionlist]

def retrievesimilarities(query):
    similarities = []
    queryembedding = model.encode(query)
    for tagquestiontup in questionembeddinglist:
        similarities.append((util.pytorch_cos_sim(queryembedding, tagquestiontup[1]).item(), tagquestiontup[0]))
    return(max(similarities))  

In [39]:
def findanswer(query):
    similaritytagtup = retrievesimilarities(query)
    tag = similaritytagtup[1]
    return answerdict[tag]

In [50]:
#evaluation
table = []

with open('intents_test.json', encoding='utf-8-sig') as testfile:
    testdata = json.load(testfile)
    for qanda in testdata['intents']:
        liste = []
        query = qanda['questions'][0]
        liste.append(query)
        predictedanswer = str(findanswer(qanda['questions'][0]))
        liste.append(predictedanswer)
        realanswer = qanda['answers'][0]
        liste.append(realanswer)
        table.append(liste)

matches = 0
fails = 0
for row in table: 
    if row[1] == row[2]:
        matches += 1
    else: 
        fails += 1

success = 100 - (fails/(fails+matches))*100
print('The bot found the fitting answer for', success, '% of the test queries.')

The bot found the fitting answer for 100.0 % of the test queries.


In [54]:
#the bot at work, to try it out, run this cell again
print('Ask me anything about alcohol addiction! Type "quit" to exit')

while True: 
    query = input('You: ')
    if query == 'quit':
        break
    else:
        print(f'\nBot: {findanswer(query)}\n')

Ask me anything about alcohol addiction! Type "quit" to exit
You: whats up

Bot: Hi, how can I help?

You: i want to stop drinking

Bot: Although stopping cold turkey is possible, for those with a severe physical dependence, it is ill-advised to do so, because alcohol’s withdrawal symptoms may include extremely unpleasant and/or dangerous physical and emotional effects that are best managed with the supervision and support of a medical detox program. Research now shows that many people quit entirely on their own without help or treatment. You may be fine to proceed on your own, but there is no harm and shame in reaching out. The symptoms of acute alcohol withdrawal include: Anxiety, cravings, jitteriness and feeling on edge, irritability, depression and trouble feeling pleasure, headache, dizziness, insomnia, fatigue, lethargy, difficulties thinking straight, concentrating, and making decisions, memory issues, nausea, vomiting, diarrhea, dehydration, decreased appetite and possible ano